In [ ]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
from transformers import AdamW
from datasets import load_dataset
import torch
from collections import Counter
#import fiftyone
import torch.nn as nn
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np
import os
import time
import glob
from typing import List, Tuple, Iterable

# uncommon features  - events of interest
# loss less compression -  sudden more bits indicates anomaly can be flagged, alerts when anomaly detected - may shift to lossy video streaming
# lossy compression of noisy data varying distortion rate - accuracy is increasing
# video to video lossy reconstruction possibility
# image frame to image frame on a need basis - human satisfaction metric, GPT based comparison, RLHF based comparison

/Users/nikilravi/Library/Python/3.9/lib/python/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/nikilravi/Library/Python/3.9/lib/python/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
# first, download and unzip dataset from here: 
# http://www.svcl.ucsd.edu/projects/anomaly/dataset.htm

In [ ]:
from PIL import Image
import torch.utils.data as data
import os
import torchvision.transforms as transforms
import torch

class UCSDAnomalyDataset(data.Dataset):
    '''
    Dataset class to load  UCSD Anomaly Detection dataset
    Input: 
    - root_dir -- directory (Train/Test) structured exactly as out-of-the-box folder downloaded from the site
    http://www.svcl.ucsd.edu/projects/anomaly/dataset.htm
    - time_stride (default 1) -- max possible time stride used for data augmentation
    - seq_len -- length of the frame sequence
    Output:
    - tensor of 10 normlized grayscale frames stiched together
    
    Note:
    [mean, std] for grayscale pixels is [0.3750352255196134, 0.20129592430286292]
    '''
    def __init__(self, root_dir, seq_len = 1, time_stride=1, transform=None):
        super(UCSDAnomalyDataset, self).__init__()
        self.root_dir = root_dir
        vids = [d for d in os.listdir(self.root_dir) if os.path.isdir(os.path.join(self.root_dir, d))]
        self.samples = []
        for d in vids:
            for t in range(1, time_stride+1):
                for i in range(1, 200):
                    if i+(seq_len-1)*t > 200:
                        break
                    self.samples.append((os.path.join(self.root_dir, d), range(i, i+(seq_len-1)*t+1, t)))
        self.pil_transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.Grayscale(),
                    transforms.ToTensor()])
        self.tensor_transform = transforms.Compose([
                    transforms.Normalize(mean=(0.3750352255196134,), std=(0.20129592430286292,))])
        
    def __getitem__(self, index):
        sample = []
        pref = self.samples[index][0]
        for fr in self.samples[index][1]:
            with open(os.path.join(pref, '{0:03d}.tif'.format(fr)), 'rb') as fin:
                frame = Image.open(fin).convert('RGB')
                frame = self.pil_transform(frame) / 255.0
                frame = self.tensor_transform(frame)
                sample.append(frame)
        sample = torch.stack(sample, axis=0)
        return sample

    def __len__(self):
        return len(self.samples)


train_ds = UCSDAnomalyDataset('/Users/nikilravi/Downloads/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train', time_stride=3)
print(train_ds.__len__())
train_dl = data.DataLoader(train_ds, batch_size=None, shuffle=True)

print(len(train_dl))

20298
20298


In [ ]:
for batch_idx, x in enumerate(train_dl):
    print(x.shape)

torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1,

KeyboardInterrupt: 

In [ ]:
# these images are grayscale- so to make them RGB, can repeat the
# across the 3 color channels
# ViT can be finetuned on grayscale

# otherwise size is already correct 224 x 224